In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from comet_ml import Experiment
from ipywidgets import interact, IntSlider

from src.data.dataset import split_dataset, tidy_plays_df
from src.data.data_query import StorageEngine
from src.data.plays_model import game_json_to_plays_list
from src.features.features import basic_features, advanced_features, normalize_plays_coords
#from src.models.xgboost import train_xgb

In [ ]:
#tidied = tidy_plays_df([2015, 2016, 2017, 2018, 2019], augment=True)
#tidied.to_csv("./data/processed/plays_2015-2020.csv", index=False)

In [ ]:
@interact(row=IntSlider(options=advanced_df.shape[0]), show_prev=[True, False], show_norm=[True, False])
def view_features(row=0, show_prev=True, show_norm=True):
    example = advanced_df.iloc[row, :].copy()
    
    plt.figure()
    plt.xlim(-100, 100)
    plt.ylim(-43, 43)
    plt.scatter(100-11, 0, c="red", label="goal", marker="<") # net
    plt.scatter(-(100-11), 0, c="red", marker=">") # net
    plt.scatter(example.x_coord, example.y_coord, c="C0", marker="+", s=200, label="true_coords")
    if show_prev:
        plt.scatter(example.previous_x_coord, example.previous_y_coord, c="C0", marker="+", s=50, label="previous_coords")
    
    if show_norm & (example.x_coord != example.x_coord_norm):
        plt.scatter(example.x_coord_norm, example.y_coord_norm, c="C1", marker="+", s=200, label="normalized_coords")
    plt.legend()
    plt.show()
    
    # print("current_event: ", example.event_type)
    # print("previous_event: ", example.previous_event_type)
    print("\nangle: ", example.angle_from_net)
    print("prev_angle: ", example.prev_angle)
    print("angle_diff: ", example.angle_from_net - example.prev_angle)
    print("\ndist: ", example.dist_from_previous)
    print("seconds: ", example.seconds_from_previous)
    print("speed: ", example.speed)

# 5 - XGBoost 

## preprocessed data

In [ ]:
season_plays_df = pd.read_csv("./data/processed/plays_2015-2020.csv", index_col=False)
train_df, test_df = split_dataset(season_plays_df)
y_train = np.where(train_df.event_type_id=="GOAL", 1, 0)
y_test = np.where(test_df.event_type_id=="GOAL", 1, 0)

In [ ]:
wpg_v_wsh = season_plays_df.loc[season_plays_df["gamePk"]==2017021065].copy()

In [ ]:
wpg_v_wsh_pre = advanced_features(wpg_v_wsh)

In [ ]:
def save_subset_df(subset_df):
    experiment = Experiment(project_name="hockey-all-star-analytics")
    experiment.log_dataframe_profile(subset_df, name="wpg_v_wsh_2017021065", dataframe_format="csv")
    experiment.end()

In [ ]:
save_subset_df(wpg_v_wsh_pre)

In [7]:
train_df

,event_idx,event_type_id,period_idx,period_type,game_time,period_time,shot_type,team_initiative_id,team_initiative_name,x_coord,...,previous_event_period,previous_event_period_time,previous_event_time,previous_event_type,previous_event_x_coord,previous_event_y_coord,gamePk,game_season,game_type,game_start_time
5394,7,SHOT,1,REGULAR,2015-10-07T23:24:23Z,00:51,Wrist Shot,TOR,Toronto Maple Leafs,-55.0,...,1.0,00:40,2015-10-07T23:24:12Z,HIT,94.0,-34.0,2015020001,20152016,R,2015-10-07T23:00:00Z
5395,9,SHOT,1,REGULAR,2015-10-07T23:24:38Z,01:05,Snap Shot,TOR,Toronto Maple Leafs,-79.0,...,1.0,00:58,2015-10-07T23:24:30Z,GIVEAWAY,-37.0,-28.0,2015020001,20152016,R,2015-10-07T23:00:00Z
5396,10,SHOT,1,REGULAR,2015-10-07T23:27:21Z,01:06,Snap Shot,TOR,Toronto Maple Leafs,-75.0,...,1.0,01:05,2015-10-07T23:24:38Z,SHOT,-79.0,0.0,2015020001,20152016,R,2015-10-07T23:00:00Z
5397,13,SHOT,1,REGULAR,2015-10-07T23:25:24Z,01:53,Wrist Shot,TOR,Toronto Maple Leafs,-55.0,...,1.0,01:35,2015-10-07T23:25:07Z,HIT,-91.0,35.0,2015020001,20152016,R,2015-10-07T23:00:00Z
5398,21,SHOT,1,REGULAR,2015-10-07T23:27:08Z,02:37,Tip-In,TOR,Toronto Maple Leafs,-61.0,...,1.0,02:32,2015-10-07T23:26:42Z,FACEOFF,-69.0,-22.0,2015020001,20152016,R,2015-10-07T23:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349169,276,SHOT,3,REGULAR,2019-04-07T04:51:22Z,15:37,Slap Shot,SJS,San Jose Sharks,67.0,...,3.0,15:04,2019-04-07T04:50:48Z,GIVEAWAY,29.0,35.0,2018021271,20182019,R,2019-04-07T02:30:00Z
349170,281,SHOT,3,REGULAR,2019-04-07T04:53:09Z,16:33,Wrist Shot,COL,Colorado Avalanche,-78.0,...,3.0,16:04,2019-04-07T04:52:14Z,FACEOFF,-69.0,-22.0,2018021271,20182019,R,2019-04-07T02:30:00Z
349171,291,SHOT,3,REGULAR,2019-04-07T04:58:03Z,18:39,Wrist Shot,COL,Colorado Avalanche,-82.0,...,3.0,18:20,2019-04-07T04:57:41Z,MISSED_SHOT,-71.0,4.0,2018021271,20182019,R,2019-04-07T02:30:00Z
349172,292,SHOT,3,REGULAR,2019-04-07T04:58:09Z,18:40,Wrist Shot,COL,Colorado Avalanche,-81.0,...,3.0,18:39,2019-04-07T04:58:03Z,SHOT,-82.0,10.0,2018021271,20182019,R,2019-04-07T02:30:00Z
